In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# DownLoad Training data from open datasets
train_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# DownLoad test data from open datasets
test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
# pass the dataset as an argument to dataloader
batch_size = 64

# create data loaders
train_dataloader = DataLoader(train_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f"shape of x [N,C,H,W] : {X.shape}")
    print(f"shape of y {y.shape} : {y.dtype}")
    break

shape of x [N,C,H,W] : torch.Size([64, 1, 28, 28])
shape of y torch.Size([64]) : torch.int64


In [5]:
# create a model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

#define a model
class NeuralNetWork(nn.Module):
    def __init__(self):
        super(NeuralNetWork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_rulu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_rulu_stack(x)
        return logits

model = NeuralNetWork().to(device)
print(model)

using cpu device
NeuralNetWork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_rulu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
# optimizizer the model parameters
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)

def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)
        pred = model(X)
        loss = loss_fn(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss,current = loss.item(),batch * len(X)
            print(f"loss {loss : >7f}  [{current : >5d}/{size : >5d}]")

def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"test error :\n Accuracy: {(100*correct) :> 0.1f},avg loss :{test_loss:>8f}")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"epoch {t+1}\n")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done")

epoch 1

loss 2.312004  [    0/60000]
loss 2.292023  [ 6400/60000]
loss 2.270930  [12800/60000]
loss 2.265259  [19200/60000]
loss 2.256935  [25600/60000]
loss 2.226278  [32000/60000]
loss 2.234724  [38400/60000]
loss 2.204657  [44800/60000]
loss 2.197436  [51200/60000]
loss 2.165191  [57600/60000]
test error :
 Accuracy:  40.6,avg loss :2.160889
epoch 2

loss 2.174416  [    0/60000]
loss 2.160447  [ 6400/60000]
loss 2.097900  [12800/60000]
loss 2.117447  [19200/60000]
loss 2.080279  [25600/60000]
loss 2.015256  [32000/60000]
loss 2.044612  [38400/60000]
loss 1.968371  [44800/60000]
loss 1.961171  [51200/60000]
loss 1.902645  [57600/60000]
test error :
 Accuracy:  55.2,avg loss :1.898030
epoch 3

loss 1.925438  [    0/60000]
loss 1.898763  [ 6400/60000]
loss 1.776909  [12800/60000]
loss 1.824742  [19200/60000]
loss 1.728251  [25600/60000]
loss 1.671804  [32000/60000]
loss 1.694566  [38400/60000]
loss 1.599117  [44800/60000]
loss 1.610733  [51200/60000]
loss 1.519655  [57600/60000]
test 

In [10]:
torch.save(model.state_dict(),"model.pth")
print("saved pytorch model state to model.pth")

saved pytorch model state to model.pth


In [22]:
model = NeuralNetWork()
model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
X,y = test_data[0][0],test_data[0][1]
with torch.no_grad():
    pred = model(X)
    print(pred[0].shape)
    print(pred[0].argmax(0).shape)
    predicted,actual = classes[pred[0].argmax(0)],classes[y]
    print(f'predicted:"{predicted}",Actual:"{actual}')

torch.Size([10])
torch.Size([])
predicted:"Ankle boot",Actual:"Ankle boot


In [20]:
for X,y in test_dataloader:
        X,y = X.to(device),y.to(device)
        pred = model(X)
        print(pred.shape)
        print(pred.argmax(1).shape)
        break

torch.Size([64, 10])
torch.Size([64])
